In [27]:

class_csv = '/home/kylecshan/saved/06_04_2019/delfTL_dist/cosine/delfTLlabel.csv'

In [28]:
import numpy as np
preds = np.array([[2, 2, 2, 4, 4, 5], [1, 1, 2, 1, 2, 5], [3, 4, 3, 5, 2, 1], [4, 1, 1, 2, 4, 4]])

In [29]:
import pandas
df = pandas.read_csv(class_csv, header=None)
array = df.values[:, :100]
# array = preds

In [30]:
print (array[1])

[   0 1479  118  305  477 1656    0  928 1504    0 1933  687 1573 1839
 1504 1773  490  450  490  713 1383  581  501  623 1557 1812 1490 1030
 1035 1672 1952  526 1766  523 1557 1586 1331 1709 1949 1860 1229  193
 1916 1264 1766  581 1979 1135  157 1921 1168 1672 1608 1766 1144 1390
  157 1315 1557 1773  980 1568 1421 1812 1873   92  934 1839 1026 1207
  782  910   44 1672 1656  610   34 1860  910  511 1911    4 1870  511
 1229  290 1979 1766 1511  639 1004 1331 1895  622  304 1419  452  105
  511 1608]


In [31]:
def mAP(array):
    N, M = array.shape
    print (N, M)
    score = 0
#     mq = 2
    mq = 20
    for i in range(N):
        running_correct = 0
        for j in range(1, M):
            if array[i][j] == array[i][0]:
                running_correct += 1
                score += running_correct/j
    score /= (N*mq)
    return score
    
print (mAP(array))
    

8000 100
0.09422041086865388


In [1]:
a = [[1,2], [2,3]]

b, c = a

print (b, c)

[1, 2] [2, 3]


In [35]:
save_path = '/home/kylecshan/saved/05_31_2019/'
N = 768

In [36]:
import faiss
import numpy as np
from sklearn.decomposition import PCA

def PCA_index(save_path, N = 768):
    index = faiss.read_index(save_path + 'tdelf.index')
    test = faiss.read_index(save_path + 'tdelftest.index')

    nb = index.ntotal
    nq = test.ntotal

    index_matrix = np.vstack([index.reconstruct(i) for i in range(nb)])
    test_matrix = np.vstack([test.reconstruct(i) for i in range(nq)])

    X = np.vstack((index_matrix, test_matrix))

    pca = PCA(n_components=N)
    pca.fit(X)  
    Y = pca.fit_transform(X) 

    index_pca = Y[:nb]
    test_pca = Y[nb:]

    index_cpu = faiss.IndexFlatL2(N)
    test_cpu = faiss.IndexFlatL2(N)

    for i in range(nb):
        index_cpu.add(np.ascontiguousarray(index_pca[i][np.newaxis, :]))

    for i in range(nq):
        test_cpu.add(np.ascontiguousarray(test_pca[i][np.newaxis, :]))

    faiss.write_index(index_cpu, save_path + 'index_pca.index')
    faiss.write_index(test_cpu, save_path + 'test_pca.index')

In [ ]:
PCA_index(save_path)